In [ ]:
import numpy as np


In [ ]:
datain = np.load('initial_inputs.npy')
dataout = np.load('initial_outputs.npy')

In [ ]:
print(datain)
print(datain.shape)   # Useful if it’s an array
print(type(datain)) 

In [ ]:
print(dataout)
print(dataout.shape)   # Useful if it’s an array
print(type(dataout)) 

# Week 1

In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from scipy.optimize import minimize
from scipy.stats import norm

# --- Initial data ---
#X_init = np.array([...])  # 30x6 array
#y_init = np.array([...])  # 30 outputs
X_init = datain
y_init = dataout

y_best = y_init.max()  # best performance so far

# --- Fit Gaussian Process ---
kernel = Matern(nu=2.5)
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-6, normalize_y=True)
gp.fit(X_init, y_init)

# --- Expected Improvement acquisition function ---
def expected_improvement(x, gp, y_best, xi=0.01):
    x = np.array(x).reshape(1, -1)
    mu, sigma = gp.predict(x, return_std=True)
    mu, sigma = mu[0], sigma[0]
    if sigma == 0.0:
        return 0.0
    imp = mu - y_best - xi
    Z = imp / sigma
    ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
    return -ei  # negative for minimization in scipy

# --- Optimize acquisition function ---
bounds = [(0,1)]*6
best_x = None
best_ei = float('inf')

for _ in range(20):  # multiple random starts
    x0 = np.random.rand(6)
    res = minimize(lambda x: expected_improvement(x, gp, y_best),
                   x0=x0, bounds=bounds, method='L-BFGS-B')
    if res.fun < best_ei:
        best_ei = res.fun
        best_x = res.x

x_next = best_x
print("Next hyperparameter set to try:", x_next)


# Week 2

In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from scipy.optimize import minimize
from scipy.stats import norm

# --- Initial data ---
#X_init = np.array([...])  # 30x6 array
#y_init = np.array([...])  # 30 outputs
X_init = datain
y_init = dataout

X_new = np.array([[0.067502, 0.634745, 0.770678, 0.925945, 0.431622, 0.542899]])
y_new = np.array([0.03701472059292424])

X_init = np.vstack([X_init, X_new])
y_init = np.append(y_init, y_new)

y_best = y_init.max()  # best performance so far

print(X_init)
print(y_init)
print(y_best)

In [ ]:
# --- Fit Gaussian Process ---
kernel = Matern(nu=2.5)
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-6, normalize_y=True)
gp.fit(X_init, y_init)
y_best = y_init.max()  # still the best observed performance


In [ ]:
def expected_improvement(x, gp, y_best, xi=0.05):
    x = np.array(x).reshape(1, -1)
    mu, sigma = gp.predict(x, return_std=True)
    mu, sigma = mu[0], sigma[0]
    if sigma == 0:
        return 0
    imp = mu - y_best - xi
    Z = imp / sigma
    ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
    return -ei  # negative for minimize


In [ ]:
bounds = [(0, 1)] * 6
best_x = None
best_ei = float('inf')

for _ in range(20):
    x0 = np.random.rand(6)
    res = minimize(lambda x: expected_improvement(x, gp, y_best),
                   x0=x0, bounds=bounds, method='L-BFGS-B')
    if res.fun < best_ei:
        best_ei = res.fun
        best_x = res.x

x_next = best_x
print("Next hyperparameter set to try:", x_next)


# Week 3

In [ ]:
this new sample is not an improvement — it’s far below the best observed score.
(a) Increase exploration slightly
You’ve now had two consecutive low-performing samples after your initial batch of 30.
(b) Use more random restarts or a global search for EI
If you’re repeatedly landing in low-output zones, your acquisition optimization might be getting trapped in local minima.

The overall procedure for selecting the next point stays the same — refit the GP and maximize Expected Improvement — but given two recent low outputs, you should now:
Increase exploration slightly (e.g. raise xi in EI),
Use more restarts or random sampling in optimizing EI,
Optionally test an alternative acquisition like UCB.

In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from scipy.optimize import minimize
from scipy.stats import norm

# --- Load initial data ---
X_init = np.load('initial_inputs.npy')   # 30x6
y_init = np.load('initial_outputs.npy')  # 30 outputs

# --- Add new observations ---
new_X = np.array([
    [0.067502, 0.634745, 0.770678, 0.925945, 0.431622, 0.542899],
    [0.400345, 0.617435, 0.564076, 0.028873, 0.730474, 0.787245]
])
new_y = np.array([0.03701472059292424, 0.21975396773107775])

X_init = np.vstack([X_init, new_X])
y_init = np.append(y_init, new_y)

# --- Update best observed performance ---
y_best = y_init.max()

# --- Fit Gaussian Process ---
kernel = Matern(nu=2.5)
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-6, normalize_y=True)
gp.fit(X_init, y_init)

# --- Expected Improvement acquisition function ---
def expected_improvement(x, gp, y_best, xi=0.05):  # increase xi for exploration
    x = np.array(x).reshape(1, -1)
    mu, sigma = gp.predict(x, return_std=True)
    mu, sigma = mu[0], sigma[0]
    if sigma == 0.0:
        return 0.0
    imp = mu - y_best - xi
    Z = imp / sigma
    ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
    return -ei  # negative for minimization

# --- Optimize acquisition function ---
bounds = [(0, 1)] * 6
best_x = None
best_ei = float('inf')

# Use more random restarts for robust optimization
n_restarts = 50
for _ in range(n_restarts):
    x0 = np.random.rand(6)
    res = minimize(lambda x: expected_improvement(x, gp, y_best, xi=0.05),
                   x0=x0, bounds=bounds, method='L-BFGS-B')
    if res.fun < best_ei:
        best_ei = res.fun
        best_x = res.x

x_next = best_x
print("Next hyperparameter set to try:", x_next)


In [ ]:
x_next_6dp = np.round(x_next, 6)
x_next_6dp
print("Next hyperparameter set to try:", x_next_6dp)

# Week 4